In [1]:
import tkinter as tk
from tkinter import ttk
from datetime import datetime
import mysql.connector

def insert_data(date, time, product, price, quantity, total_sales, total_profit):
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="11111111",
            database="sales_data"
        )
        cursor = connection.cursor()

        total_sales = float(total_sales.replace("₹ ", "").replace(",", ""))
        total_profit = float(total_profit.replace("₹ ", "").replace(",", ""))

        insert_query = "INSERT INTO sales_data (date, time, product, price, quantity, total_sales, total_profit) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        data = (date, time, product, price, quantity, total_sales, total_profit)

        cursor.execute(insert_query, data)
        connection.commit()
        print("Data inserted successfully!")
    except mysql.connector.Error as error:
        print(f"Error: {error}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

def update_date_time():
    current_datetime = datetime.now()
    date_str, time_str = current_datetime.strftime("%Y-%m-%d %H:%M:%S").split()
    entries_frame1[0].set(date_str)
    entries_frame1[1].set(time_str)
    root.after(1000, update_date_time)

def update_price_field(event=None):
    product_name = product_combobox.get()
    product_price = float(products.get(product_name, 0))
    price = f"₹ {product_price:.2f}"  # Update product_price here
    price_entry.config(state="normal")
    price_entry.delete(0, tk.END)
    price_entry.insert(0, price)
    price_entry.config(state="readonly")
    calculate_sales_and_profit()  # Call this to update Total Sales and Total Profit

def calculate_sales_and_profit(event=None):
    product_name = product_combobox.get()
    product_price = float(products.get(product_name, 0))
    quantity = quantity_entry.get()
    if quantity:
        quantity = int(quantity)
        total_sales = product_price * quantity
        total_profit = total_sales * 0.08
        total_sales_var.set(f"₹ {total_sales:.2f}")
        total_profit_var.set(f"₹ {total_profit:.2f}")
    else:
        total_sales_var.set("Enter Quantity")
        total_profit_var.set("Enter Quantity")

def insert_data_button_click():
    calculate_sales_and_profit()  # Recalculate total sales and total profit
    product_name = product_combobox.get()
    product_price = float(products.get(product_name, 0))  # Calculate the product price based on the selected product
    insert_data(entries_frame1[0].get(), entries_frame1[1].get(), product_name, product_price, int(quantity_entry.get()), total_sales_var.get(), total_profit_var.get())
    update_data_table()

def update_data_table():
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="11111111",
            database="sales_data"
        )
        cursor = connection.cursor()

        # Fetch data in descending order of the primary key (change 'id' to your actual primary key field)
        cursor.execute("SELECT date, time, product, price, quantity, total_sales, total_profit FROM sales_data ORDER BY id DESC")
        data = cursor.fetchall()

        # Clear the existing widgets in frame5
        for child in frame5.winfo_children():
            child.grid_forget()

        # Create a Treeview widget for displaying data without the blank column
        tree = ttk.Treeview(frame5, columns=("Date", "Time", "Product", "Price", "Quantity", "Total Sales", "Total Profit"), height=25)  # Increase the height here
        tree.column("#0", width=0, stretch=tk.NO)  # Hide the first column
        tree.heading("#1", text="Date")
        tree.heading("#2", text="Time")
        tree.heading("#3", text="Product")
        tree.heading("#4", text="Price")
        tree.heading("#5", text="Quantity")
        tree.heading("#6", text="Total Sales")
        tree.heading("#7", text="Total Profit")

        # Center-align the data
        tree.tag_configure('center', anchor='center')
        for col in tree["columns"]:
            tree.heading(col, anchor='center')
            tree.column(col, anchor='center')

        tree.grid(row=0, column=0, sticky='nsew')

        for row in data:
            tree.insert("", "end", values=row)

    except mysql.connector.Error as error:
        print(f"Error: {error}")
    finally:
        if connection is not None and connection.is_connected():
            cursor.close()
            connection.close()

# Create the main tkinter window
root = tk.Tk()
root.geometry("800x700")
root.title("Sales Data Entry")

# 1st Part: Date and Time, Product Details, Total Sales, and Total Profit
frame1 = ttk.LabelFrame(root, text="Date and Time", width=300)
frame1.grid(row=0, column=0, padx=10, pady=10, sticky='nsew')
frame2 = ttk.LabelFrame(root, text="Product Details", width=300)
frame2.grid(row=0, column=1, padx=10, pady=10, sticky='nsew')
frame3 = ttk.LabelFrame(root, text="Total Sales and Total Profit", width=300)
frame3.grid(row=0, column=2, padx=10, pady=10, sticky='nsew')

# 2nd Part: Insert Data
frame4 = ttk.LabelFrame(root, text="Insert Data")
frame4.grid(row=1, column=0, columnspan=3, padx=10, pady=10, sticky='nsew')

# 3rd Part: Show Data Table
frame5 = ttk.LabelFrame(root, text="Show Data Table")
frame5.grid(row=2, column=0, columnspan=3, padx=10, pady=10, sticky='nsew')

entry_width = 22

# Labels and entry fields for Date and Time
labels_frame1 = ["Date:", "Time:"]
entries_frame1 = [tk.StringVar() for _ in labels_frame1]

# Default date and time values
current_datetime = datetime.now()
date_str, time_str = current_datetime.strftime("%Y-%m-%d %H:%M:%S").split()
entries_frame1[0].set(date_str)
entries_frame1[1].set(time_str)

for i, label_text in enumerate(labels_frame1):
    label = ttk.Label(frame1, text=label_text, foreground="blue")
    label.grid(row=i, column=0, padx=5, pady=5)
    entry = ttk.Entry(frame1, width=entry_width, textvariable=entries_frame1[i], state="readonly")
    entry.grid(row=i, column=1, padx=5, pady=5)

# Create a dictionary to store product prices
products = {}
with open("products.txt", "r") as file:
    for line in file:
        parts = line.strip().split(":")
        if len(parts) == 2:
            product_name, product_price = parts[0], parts[1]
            products[product_name] = product_price

product_names = list(products.keys())

# Product details section
product_label = ttk.Label(frame2, text="Product:", foreground="blue")
product_label.grid(row=0, column=0, padx=5, pady=5)
product_combobox = ttk.Combobox(frame2, values=product_names, width=entry_width)
product_combobox.grid(row=0, column=1, padx=5, pady=5)

# Bind events to update the price and calculate sales/profit
product_combobox.bind("<<ComboboxSelected>>", update_price_field)

price_label = ttk.Label(frame2, text="Price:", foreground="blue")
price_label.grid(row=1, column=0, padx=5, pady=5)
price_entry = ttk.Entry(frame2, width=entry_width, state="readonly")
price_entry.grid(row=1, column=1, padx=5, pady=5)

quantity_label = ttk.Label(frame2, text="Quantity:", foreground="blue")
quantity_label.grid(row=2, column=0, padx=5, pady=5)
quantity_entry = ttk.Entry(frame2, width=entry_width)
quantity_entry.grid(row=2, column=1, padx=5, pady=5)

# Bind events to calculate sales and profit
product_combobox.bind("<KeyRelease>", calculate_sales_and_profit)
quantity_entry.bind("<KeyRelease>", calculate_sales_and_profit)

# Total Sales and Total Profit section
total_sales_label = ttk.Label(frame3, text="Total Sales:", foreground="blue")
total_sales_label.grid(row=0, column=0, padx=5, pady=5)
total_sales_var = tk.StringVar()
total_sales_entry = ttk.Entry(frame3, textvariable=total_sales_var, state="readonly")
total_sales_entry.grid(row=0, column=1, padx=5, pady=5)

total_profit_label = ttk.Label(frame3, text="Total Profit:", foreground="blue")
total_profit_label.grid(row=1, column=0, padx=5, pady=5)
total_profit_var = tk.StringVar()
total_profit_entry = ttk.Entry(frame3, textvariable=total_profit_var, state="readonly")
total_profit_entry.grid(row=1, column=1, padx=5, pady=5)

# Buttons for Insert Data
insert_data_button = ttk.Button(frame4, text="Insert Data", command=insert_data_button_click)
insert_data_button.grid(row=0, column=0, padx=10, pady=10, sticky='nsew')

# Configure column weights for root to make the frames fill the entire width
root.columnconfigure(0, weight=1)
root.columnconfigure(1, weight=1)
root.columnconfigure(2, weight=1)

# Configure row weights for root to make the frames fill the entire height
root.rowconfigure(0, weight=1)
root.rowconfigure(1, weight=1)
root.rowconfigure(2, weight=1)

# Configure the inner frames as you already did
frame1.columnconfigure(0, weight=1)
frame2.columnconfigure(0, weight=1)
frame3.columnconfigure(0, weight=1)
frame4.columnconfigure(0, weight=1)
frame5.columnconfigure(0, weight=1)

frame5.rowconfigure(0, weight=1)

# Update date and time
update_date_time()

# Initially update the data table
update_data_table()

# Function to update the data table at regular intervals
def update_data_table_periodically():
    update_data_table()
    root.after(60000, update_data_table_periodically)  # Update every 60 seconds (adjust as needed)

# Start updating the data table periodically
update_data_table_periodically()

# Start the main tkinter event loop
root.mainloop()

Data inserted successfully!
